# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("Resources/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dunedin,-45.8742,170.5036,53.80,92,100,4.00,NZ,1671592195
1,1,naze,28.3667,129.4833,64.45,69,100,15.68,JP,1671592324
2,2,rocha,-34.4833,-54.3333,58.46,94,0,5.70,UY,1671592325
3,3,iqaluit,63.7506,-68.5145,-2.31,77,92,10.00,CA,1671592326
4,4,upernavik,72.7868,-56.1549,9.05,96,100,14.56,GL,1671592326


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot

gmaps.configure(api_key= g_key)
heatMapBase = gmaps.figure(center= (0, 0), zoom_level= 2)
heatMapLayer = gmaps.heatmap_layer(city_data_df[["Lat", "Lng"]], weights= city_data_df["Humidity"], max_intensity= 100, point_radius= 15)
heatMapBase.add_layer(heatMapLayer)

# Display the map
heatMapBase

Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# (I set the below values to filter city_data down to 20 rows to limit api usage)

vacation_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= 90) &
                                       (city_data_df["Max Temp"] >= 60) &
                                       (city_data_df["Humidity"] >= 50) &
                                       (city_data_df["Cloudiness"] <= 20) &
                                       (city_data_df["Wind Speed"] <= 5)                                     
                                       ]

# Drop any rows with null values
vacation_cities_df.dropna()

# Display sample data
vacation_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,grand baie,-20.0182,57.5802,78.46,67,8,3.00,MU,1671592045
92,92,cabo san lucas,22.8909,-109.9124,72.52,65,20,4.61,MX,1671592324
212,212,isangel,-19.5500,169.2667,84.88,65,8,4.27,VU,1671592454
216,216,jamiltepec,16.2833,-97.8167,68.34,78,0,3.06,MX,1671592457
227,227,vredendal,-31.6683,18.5012,68.45,58,0,4.70,ZA,1671592464


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
9,grand baie,MU,-20.0182,57.5802,67,
92,cabo san lucas,MX,22.8909,-109.9124,65,
212,isangel,VU,-19.5500,169.2667,65,
216,jamiltepec,MX,16.2833,-97.8167,78,
227,vredendal,ZA,-31.6683,18.5012,58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
#radius = # YOUR CODE HERE
params = {
    "type": "lodging",
    "rankby": "distance",
    "key": g_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    # params["filter"] = # YOUR CODE HERE
    # params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params= params).json()

    # extract results
    results = name_address['results']
    
    # Convert the API response to JSON format
    # name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
grand baie - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
isangel - nearest hotel: No hotel found
jamiltepec - nearest hotel: No hotel found
vredendal - nearest hotel: No hotel found
acapulco - nearest hotel: No hotel found
mecca - nearest hotel: No hotel found
puerto escondido - nearest hotel: No hotel found
acajutla - nearest hotel: No hotel found
coquimbo - nearest hotel: No hotel found
bachaquero - nearest hotel: No hotel found
west bay - nearest hotel: No hotel found
nizwa - nearest hotel: No hotel found
otjimbingwe - nearest hotel: No hotel found
walvis bay - nearest hotel: No hotel found
los palacios - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
9,grand baie,MU,-20.0182,57.5802,67,No hotel found
92,cabo san lucas,MX,22.8909,-109.9124,65,No hotel found
212,isangel,VU,-19.5500,169.2667,65,No hotel found
216,jamiltepec,MX,16.2833,-97.8167,78,No hotel found
227,vredendal,ZA,-31.6683,18.5012,58,No hotel found
243,acapulco,MX,16.8634,-99.8901,73,No hotel found
263,mecca,SA,21.4267,39.8261,61,No hotel found
290,puerto escondido,MX,15.8500,-97.0667,69,No hotel found
332,acajutla,SV,13.5928,-89.8275,70,No hotel found
382,coquimbo,CL,-29.9533,-71.3436,82,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
# Configure the map plot
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(0, 0), zoom_level= 2)

# Add the humidity layer
heat_layer = gmaps.heatmap_layer(city_data_df[["Lat", "Lng"]], weights= city_data_df["Humidity"], dissipating= True, max_intensity= 100, point_radius= 15)

# Add the hotel markers layer
marker_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)

# Display the map
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# locations = hotel_df[["Lat", "Lng"]]